In [1]:
# !pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from keras.models import load_model
from datetime import datetime
tf.enable_eager_execution()

C:\Users\Affinity\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W1024 14:43:12.257216 14744 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),    
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 32)         

In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).batch(64) # 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [8]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_acc',
    save_best_only=True)

In [9]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [10]:
history = model.fit(train_data,
                    epochs=10,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

W1024 14:44:13.171307 14744 deprecation.py:323] From C:\Users\Affinity\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 0 ends at 14:50:47.593855: {'loss': 1.7538973316192628, 'acc': 0.423425, 'val_loss': 1.4163781693027278, 'val_acc': 0.4933}
Epoch 1 ends at 14:57:26.374454: {'loss': 1.1922735773086548, 'acc': 0.580475, 'val_loss': 1.1919147076120802, 'val_acc': 0.583}
Epoch 2 ends at 15:04:07.705772: {'loss': 1.0225573006629944, 'acc': 0.64025, 'val_loss': 1.230954049119524, 'val_acc': 0.5635}
Epoch 3 ends at 15:10:43.338337: {'loss': 0.9361002067565918, 'acc': 0.66855, 'val_loss': 1.0627076986489024, 'val_acc': 0.6219}
Epoch 4 ends at 15:17:18.990613: {'loss': 0.8761671788692474, 'acc': 0.6927, 'val_loss': 0.965434617677312, 'val_acc': 0.6594}
Epoch 5 ends at 15:24:00.291806: {'loss': 0.8228369772911072, 'acc': 0.7107, 'val_loss': 0.9002092731226782, 'val_acc': 0.6809}
Epoch 6 ends at 15:30:38.501201: {'loss': 0.7946673872947693, 'acc': 0.72055, 'val_loss': 0.968529917631939, 'val_acc': 0.6654}
Epoch 7 ends at 15:37:23.631113: {'loss': 0.7666522501468659, 'acc': 0.7294, 'val_loss': 1.0548914010

In [11]:
model.evaluate(test_data)

    157/Unknown - 0s 58ms/step - loss: 0.7984 - acc: 0.718 - 0s 56ms/step - loss: 0.9168 - acc: 0.695 - 0s 57ms/step - loss: 0.9136 - acc: 0.677 - 0s 56ms/step - loss: 0.8498 - acc: 0.695 - 0s 55ms/step - loss: 0.8475 - acc: 0.700 - 0s 55ms/step - loss: 0.8406 - acc: 0.703 - 0s 55ms/step - loss: 0.8446 - acc: 0.700 - 0s 54ms/step - loss: 0.8485 - acc: 0.701 - 0s 54ms/step - loss: 0.8446 - acc: 0.701 - 1s 54ms/step - loss: 0.8585 - acc: 0.700 - 1s 55ms/step - loss: 0.8619 - acc: 0.694 - 1s 55ms/step - loss: 0.8803 - acc: 0.692 - 1s 55ms/step - loss: 0.8682 - acc: 0.694 - 1s 55ms/step - loss: 0.8641 - acc: 0.695 - 1s 55ms/step - loss: 0.8579 - acc: 0.700 - 1s 55ms/step - loss: 0.8707 - acc: 0.696 - 1s 55ms/step - loss: 0.8670 - acc: 0.696 - 1s 55ms/step - loss: 0.8754 - acc: 0.691 - 1s 56ms/step - loss: 0.8785 - acc: 0.689 - 1s 56ms/step - loss: 0.8940 - acc: 0.683 - 1s 56ms/step - loss: 0.8814 - acc: 0.687 - 1s 56ms/step - loss: 0.8799 - acc: 0.691 - 1s 56ms/step - loss: 0.8787 - acc: 0

[0.9169710684733786, 0.6766]

In [12]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))

In [13]:
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 32)         

In [14]:
m.evaluate(test_data)

    157/Unknown - 0s 203ms/step - loss: 0.7984 - acc: 0.71 - 0s 128ms/step - loss: 0.9168 - acc: 0.69 - 0s 103ms/step - loss: 0.9136 - acc: 0.67 - 0s 90ms/step - loss: 0.8498 - acc: 0.6953 - 0s 83ms/step - loss: 0.8475 - acc: 0.700 - 0s 78ms/step - loss: 0.8406 - acc: 0.703 - 1s 75ms/step - loss: 0.8446 - acc: 0.700 - 1s 72ms/step - loss: 0.8485 - acc: 0.701 - 1s 71ms/step - loss: 0.8446 - acc: 0.701 - 1s 69ms/step - loss: 0.8585 - acc: 0.700 - 1s 68ms/step - loss: 0.8619 - acc: 0.694 - 1s 67ms/step - loss: 0.8803 - acc: 0.692 - 1s 66ms/step - loss: 0.8682 - acc: 0.694 - 1s 65ms/step - loss: 0.8641 - acc: 0.695 - 1s 65ms/step - loss: 0.8579 - acc: 0.700 - 1s 64ms/step - loss: 0.8707 - acc: 0.696 - 1s 64ms/step - loss: 0.8670 - acc: 0.696 - 1s 63ms/step - loss: 0.8754 - acc: 0.691 - 1s 63ms/step - loss: 0.8785 - acc: 0.689 - 1s 63ms/step - loss: 0.8940 - acc: 0.683 - 1s 63ms/step - loss: 0.8814 - acc: 0.687 - 1s 63ms/step - loss: 0.8799 - acc: 0.691 - 1s 63ms/step - loss: 0.8787 - acc: 

[0.9169710684733786, 0.6766]

In [15]:
m.evaluate(valid_data)

    157/Unknown - 0s 237ms/step - loss: 1.1088 - acc: 0.59 - 0s 147ms/step - loss: 0.8076 - acc: 0.71 - 0s 117ms/step - loss: 0.8249 - acc: 0.70 - 0s 101ms/step - loss: 0.7909 - acc: 0.72 - 0s 92ms/step - loss: 0.8076 - acc: 0.7312 - 1s 86ms/step - loss: 0.8186 - acc: 0.716 - 1s 81ms/step - loss: 0.8124 - acc: 0.721 - 1s 78ms/step - loss: 0.8182 - acc: 0.709 - 1s 76ms/step - loss: 0.8082 - acc: 0.715 - 1s 74ms/step - loss: 0.8231 - acc: 0.710 - 1s 72ms/step - loss: 0.8483 - acc: 0.704 - 1s 71ms/step - loss: 0.8506 - acc: 0.701 - 1s 70ms/step - loss: 0.8382 - acc: 0.704 - 1s 69ms/step - loss: 0.8424 - acc: 0.704 - 1s 68ms/step - loss: 0.8624 - acc: 0.696 - 1s 67ms/step - loss: 0.8630 - acc: 0.699 - 1s 66ms/step - loss: 0.8585 - acc: 0.700 - 1s 66ms/step - loss: 0.8604 - acc: 0.699 - 1s 66ms/step - loss: 0.8540 - acc: 0.702 - 1s 65ms/step - loss: 0.8470 - acc: 0.703 - 1s 65ms/step - loss: 0.8474 - acc: 0.706 - 1s 65ms/step - loss: 0.8485 - acc: 0.707 - 1s 64ms/step - loss: 0.8437 - acc: 

[0.9085101060047271, 0.6873]